In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import matplotlib.animation as animation
from IPython.display import HTML
import numpy as np
import json
import ast
import pandas as pd
import os
from scipy import ndimage
from skimage import io
from skimage import transform

In [ ]:
images_analytics = PATH_TO_ANALYTICS_IMG
labels_analytics_poly = PATH_TO_ANALYTICS_JSON_POLY #shape: bitmap + origin point
labels_analytics_people = PATH_TO_ANALYTICS_JSON_PEOPLE #shape: bounding box left upper and right lower point

In [ ]:
    #outpus CSV with ID column corresponding to each new instance of a task
    #timestamp and task level table
def analytics_to_dataframe(images_analytics, labels_analytics_poly, labels_analytics, people):
    sorted_images = [images_analytics+image for image in sorted(os.listdir(images_analytics))]
    sorted_labels_poly = [labels_analytics_poly+label for label in sorted(os.listdir(labels_analytics_poly))]
    sorted_labels_people = [labels_analytics_people+label for label in sorted(os.listdir(labels_analytics_people))]


    analytics_time_serie = pd.DataFrame(columns = ['date','time','task', 'position','data', 'ID', 'workers'])

    #to keep track of ongoing task to increment ID for each new task
    tasks = {'Concrete_pump_hose':[0,False],
                       'Horizontal formwork_model': [0, False],
                       'Rebars_model': [0, False],
                       'Vertical formwork_model': [0, False],
                        'Rebars':[0, False],
                      'Horizontal_formwork': [0, False],
                      'Vertical_formwork': [0, False], 
            } #[task number, task is ongoing]


    #for each frame
    for image, label, people_label in tqdm(zip(sorted_images, sorted_labels_poly, sorted_labels_people)):
        if '.ipynb' in label:
            continue

        date, time = image.split('/')[-1][:10], image.split('/')[-1][11:-4]
        objects = ast.literal_eval(open(label).read())['objects']

        frame_tasks = []
        for obj in objects:
            frame_tasks.append(obj['classTitle'])
            if tasks[obj['classTitle']][1] == False: #if such a task was not ongoing
                tasks[obj['classTitle']][1] = True #set it as ongoing
                tasks[obj['classTitle']][0] +=1

        for task in tasks.keys():
            if task not in frame_tasks: #if a task is not in current frame, then it has stopped
                tasks[task][1] = False


        #each object in a frame
        for obj in objects:
            workers = 0
            task = obj['classTitle']
            ID = task+'_'+str(tasks[task][0])
            position = (obj['bitmap']['origin'][0], obj['bitmap']['origin'][1])
            bitmap = obj['bitmap']['data']

            #nb workers
            people_objects = ast.literal_eval(open(people_label).read())['objects']
            for people in people_objects:
                if people_in_bitmap(people['points']['exterior'], position, bitmap):
                    workers += 1
            analytics_time_serie = analytics_time_serie.append(pd.DataFrame([[date, time, task, position, bitmap, ID, workers]], 
                                                                                columns =  analytics_time_serie.columns)

                                                                )


    return analytics_time_serie